In [7]:
# Libraries
import os
import re
import json
import time
import random
import pandas as pd

from bs4 import BeautifulSoup

from prettytable import from_csv

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

Current working directory: c:\Users\FMeic\data_science\ADS_Project_group3


In [10]:
# Set options for Chrome-Driver (headless mode)
opts = Options()
#opts.add_argument("--headless")
#opts.add_argument("user-agent=" + random.choice(ua_list))
service = Service(executable_path=r'C:\Tools\chromedriver\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=opts)
# Chrome-Driver
driver = webdriver.Chrome(service=service, options=opts)

# Website to scrape
url = 'https://www.immoscout24.ch/de/immobilien/mieten/kanton-zuerich?pn=1'
driver.get(url)

# Get webpage number from url
pattern = r'=(.*)'
page = re.search(pattern, url)

# Get the full content of the website
source = driver.page_source

# Close driver
driver.close()

# Parse HTML content with BeautifulSoup
soup = BeautifulSoup(source, 'html.parser')

# Get links
links = []
link_elements = soup.find_all(class_='css-qqdlpa')
for element in link_elements:
    links.append('https://www.autoscout24.ch' + element['href'])

# Get types
types_elements = soup.find_all(class_='chakra-heading css-svmyrf')
types = [element.text for element in types_elements]

# Get prices
prices_elements = soup.find_all(class_='chakra-text css-bwl0or')
prices = [element.text for element in prices_elements]

# Dataframe
df = pd.DataFrame({ 'Page': page.group(1),
                    'Type': types,
                    'Price': prices,
                    'Weblink': links})

# Save to file
df.to_csv('autoscout24.csv', sep=";", index=False)

# Show file
with open('autoscout24.csv') as table_file:
    tab = from_csv(table_file, delimiter=';')
    
    # Set the alignment for all columns to 'l' (left-align)
    for field_name in tab.field_names:
        tab.align[field_name] = 'l'
    
    print(tab)

+------+------+-------+---------+
| Page | Type | Price | Weblink |
+------+------+-------+---------+
+------+------+-------+---------+


In [1]:
print("hi")

hi
